In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from geopy.distance import distance, lonlat
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
from ast import literal_eval
import re
import pickle

# Preprocessing Earthquake probabilities

In [ ]:
# Creating the DataFrame of the earthquakes' probabilities
df = pd.read_table('../incremental_annual_rates/incremental_annual_rates_italy.txt', sep='\s+', header=None)

df.columns = ['Lat', 'Long'] + [str(np.around(x,1)) for x in np.arange(4.6, 8.7, 0.1)]
df.drop(df.columns[42:], axis=1, inplace=True)
df.fillna(value=0.0, inplace=True)
df = df[df['4.6'] != 0.0]
df.rename(columns={'Lat':'Long', 'Long':'Lat'}, inplace=True)

df.to_csv('incremental_annual_rates/incremental_annual_rates_italy_without_zeros.csv', index=False)

In [2]:
df = pd.read_csv('../incremental_annual_rates/incremental_annual_rates_USA.csv')
#df.set_index(['Lat','Long'], inplace=True)
df.head(2)

,Lat,Long,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,...,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6
0,-82.0,24.5,5.522443e-09,4.386632e-09,3.484426e-09,2.767778e-09,2.198524e-09,1.746350e-09,1.387175e-09,1.101872e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-81.9,24.5,7.100443e-09,5.640082e-09,4.480077e-09,3.558651e-09,2.826737e-09,2.245357e-09,1.783551e-09,1.416725e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df = df[df['4.6'] != 0.0]

In [12]:
df.drop(labels=['8.5', '8.6'], axis=1, inplace=True)

In [27]:
df.describe()

,Lat,Long,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,...,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4
count,81992.000000,81992.000000,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,...,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04,8.199200e+04
mean,-99.294077,39.397524,2.885112e-05,2.375665e-05,1.956876e-05,1.612474e-05,1.329136e-05,1.095947e-05,9.039583e-06,7.458347e-06,...,1.115545e-07,8.926562e-08,7.424791e-08,6.175672e-08,5.136700e-08,4.272521e-08,3.553728e-08,2.955863e-08,2.458579e-08,2.044957e-08
std,13.802657,5.248633,7.037414e-05,5.839796e-05,4.847213e-05,4.024265e-05,3.341734e-05,2.775491e-05,2.305592e-05,1.915547e-05,...,3.320603e-07,2.766990e-07,2.301482e-07,1.914289e-07,1.592236e-07,1.324365e-07,1.101558e-07,9.162364e-08,7.620923e-08,6.338807e-08
min,-124.800000,24.500000,2.989997e-11,2.375039e-11,1.886561e-11,1.498549e-11,1.190339e-11,9.455202e-12,7.510534e-12,5.965829e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-110.600000,35.300000,1.134513e-06,9.050906e-07,7.203464e-07,5.743607e-07,4.579084e-07,3.650919e-07,2.911420e-07,2.318380e-07,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,-99.500000,39.600000,5.312655e-06,4.255761e-06,3.405683e-06,2.727693e-06,2.182770e-06,1.749890e-06,1.401403e-06,1.124050e-06,...,6.836078e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,-88.600000,43.700000,1.790731e-05,1.442202e-05,1.168117e-05,9.442183e-06,7.627315e-06,6.152295e-06,4.967933e-06,4.013171e-06,...,3.358564e-08,1.335146e-08,1.110526e-08,9.236951e-09,7.682961e-09,6.390409e-09,5.315311e-09,4.421083e-09,3.677296e-09,3.058642e-09
max,-66.900000,49.300000,7.809130e-04,6.495351e-04,5.402598e-04,4.493685e-04,3.737685e-04,3.108871e-04,2.585846e-04,2.150813e-04,...,3.737684e-06,3.108871e-06,2.585846e-06,2.150813e-06,1.788968e-06,1.487999e-06,1.237664e-06,1.029444e-06,8.562541e-07,7.122011e-07


In [14]:
df.to_csv('earthquake_probabilities/usa.csv', index=False)

In [18]:
df = pd.read_csv('earthquake_probabilities/usa.csv')

In [29]:
res = 2
ds = df.copy()
ds['Lat'] = (ds['Lat']*res).astype(int)
ds['Long'] = (ds['Long']*res).astype(int)
ds = ds.groupby(['Lat','Long']).sum()
ds = ds[ds['4.6'] != 0.0]
ds.reset_index(inplace=True)
ds['Lat'] = ds['Lat']/res
ds['Long'] = ds['Long']/res
ds

,Lat,Long,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,...,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4
0,-124.5,42.0,0.000068,0.000056,0.000047,0.000039,0.000032,0.000027,0.000022,0.000019,...,3.234575e-07,2.690403e-07,2.237780e-07,1.861304e-07,1.548165e-07,1.287708e-07,1.071069e-07,8.908760e-08,7.409984e-08,6.163356e-08
1,-124.5,42.5,0.000270,0.000225,0.000187,0.000155,0.000129,0.000107,0.000089,0.000074,...,1.292297e-06,1.074886e-06,8.940510e-07,7.436392e-07,6.185322e-07,5.144727e-07,4.279197e-07,3.559281e-07,2.960481e-07,2.462421e-07
2,-124.5,43.0,0.000056,0.000046,0.000038,0.000032,0.000027,0.000022,0.000018,0.000015,...,2.657948e-07,2.210785e-07,1.838851e-07,1.529489e-07,1.272174e-07,1.058148e-07,8.801292e-08,7.320596e-08,6.089006e-08,5.064615e-08
3,-124.5,47.5,0.000096,0.000080,0.000067,0.000055,0.000046,0.000038,0.000032,0.000027,...,4.615618e-07,3.839104e-07,3.193227e-07,2.656011e-07,2.209173e-07,1.837510e-07,1.528375e-07,1.271247e-07,1.057377e-07,8.794878e-08
4,-124.5,48.0,0.000366,0.000305,0.000253,0.000211,0.000175,0.000146,0.000121,0.000101,...,1.753517e-06,1.458512e-06,1.213137e-06,1.009044e-06,8.392859e-07,6.980876e-07,5.806440e-07,4.829586e-07,4.017075e-07,3.341257e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,-67.5,46.5,0.000108,0.000086,0.000068,0.000054,0.000043,0.000034,0.000027,0.000022,...,1.358749e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3499,-67.5,47.0,0.000035,0.000028,0.000022,0.000017,0.000014,0.000011,0.000009,0.000007,...,4.379160e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3500,-67.0,44.5,0.000201,0.000160,0.000127,0.000101,0.000080,0.000064,0.000051,0.000040,...,2.536172e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3501,-67.0,45.0,0.000085,0.000068,0.000054,0.000043,0.000034,0.000027,0.000021,0.000017,...,1.070309e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
ds.to_csv('earthquake_probabilities/usa_ds23.csv', index=False)

# Preprocessing the network

In [36]:
def read_lgf_to_networkx_extended(lgf_file):
    file = open(lgf_file, 'r')
    all_line = file.read().split('\n')
    # edge: u v label onspine unavInit unavFinal
    all_match = [re.findall(r'^(\d+)\t\((.+),(.+)\)|^(\d+)\t(\d+)\t(\d+)\t*(\d*)\t*(\S*)\t*(\S*)|(^\d+-\d+( \d+-\d+)+)', line) for line in all_line]
    all_match = filter(len, all_match)
    G = nx.MultiGraph()
    SRLGs = []
    for line in all_match:
        line = line[0]
        if line[0]:
            #G.add_node(int(line[0]), pos=(float(line[1]),float(line[2])))
            G.add_node(int(line[0]), Longitude=float(line[1]), Latitude=float(line[2]))
        elif line[3]:
            u, v = int(line[3]), int(line[4])
            G.add_edge(u,v)
            G[u][v][0]['points'] = {'point': [
                {'Longitude': G.nodes[u]['Longitude'], 'Latitude': G.nodes[u]['Latitude']},
                {'Longitude': G.nodes[v]['Longitude'], 'Latitude': G.nodes[v]['Latitude']}]}
            if line[8]:
                onspine = int(line[6])
                unav_1 = float(line[7])
                unav_k = float(line[8])
                G[u][v][0]['onspine'] = onspine
                G[u][v][0]['unav1'] = unav_1
                G[u][v][0]['unav'] = unav_k
            elif line[7]:
                onspine = int(line[6])
                unav = float(line[7])
                G[u][v][0]['onspine'] = onspine
                G[u][v][0]['unav1'] = unav
                G[u][v][0]['unav'] = unav
        if line[9]:
            srlg = line[9]
            SRLGs.append([ tuple(map(int, link.split('-'))) for link in srlg.split()])
    if SRLGs:
        return G, SRLGs
    else:
        return G

In [2]:
g = nx.read_gml('networks/italy_995.gml', label='id')
#g = read_lgf_to_networkx_extended('networks/usa_99.lgf')

In [65]:
def intensity_europe(M, R):
    h = 3.91
    D = np.sqrt(h**2 + R**2)
    return 1.621*M - 1.343 - 0.0086*(D-h) - 1.037*(np.log(D)-np.log(h))

def intensity_usa(M, R):
    h = 3.91
    D = np.sqrt(h**2 + R**2)
    return 0.44 + 1.70*M - 0.0048*D - 2.73*np.log10(D)

In [31]:
l = [tuple(d.values()) for d in g.edges[(0, 28, 0)]['points']['point']]
edge = LineString(l)
point = Point(10.8,48.3)
nearest_point = nearest_points(edge, point)[0]
nearest_point.coords[:]

[(8.616667, 44.91666)]

In [67]:
nx.write_gml(g, 'networks/usa_99.gml')

In [ ]:
for edge in g.edges:
    e = LineString([tuple(d.values()) for d in g.edges[edge]['points']['point']])
    p = list(e.coords)
    l = 0
    for x,y in zip(p[:-1], p[1:]):
        l += distance(lonlat(*x), lonlat(*y)).km
    g.edges[edge]['length'] = l
    print(edge, l)

In [ ]:
for e in g.edges:
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    edge = LineString(edge_points)
    epicenter = Point(10.5,48.2)
    nearest_point = nearest_points(edge, epicenter)[0].coords[0]
    print(nearest_point)
    print(epicenter.coords[0])
    dist = distance(lonlat(*epicenter.coords[0]), lonlat(*nearest_point)).km
    intensity = intensity_europe(8.6, dist)
    print("%.1f" % intensity)

# Earthquake intensity calculation

In [68]:
df = pd.read_csv('earthquake_probabilities/usa_ds23.csv')

In [69]:
#df['Epicenter'] = list(zip(df.Lat, df.Long))
df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)
df.drop(['Lat', 'Long'], axis=1, inplace=True)

In [70]:
#df=df_4
df.head()

,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,5.4,5.5,...,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4
Epicenter,,,,,,,,,,,,,,,,,,,,,
"(-124.5, 42.0)",0.000068,0.000056,0.000047,0.000039,0.000032,0.000027,0.000022,0.000019,0.000015,0.000013,...,3.234575e-07,2.690403e-07,2.237780e-07,1.861304e-07,1.548165e-07,1.287708e-07,1.071069e-07,8.908760e-08,7.409984e-08,6.163356e-08
"(-124.5, 42.5)",0.000270,0.000225,0.000187,0.000155,0.000129,0.000107,0.000089,0.000074,0.000062,0.000051,...,1.292297e-06,1.074886e-06,8.940510e-07,7.436392e-07,6.185322e-07,5.144727e-07,4.279197e-07,3.559281e-07,2.960481e-07,2.462421e-07
"(-124.5, 43.0)",0.000056,0.000046,0.000038,0.000032,0.000027,0.000022,0.000018,0.000015,0.000013,0.000011,...,2.657948e-07,2.210785e-07,1.838851e-07,1.529489e-07,1.272174e-07,1.058148e-07,8.801292e-08,7.320596e-08,6.089006e-08,5.064615e-08
"(-124.5, 47.5)",0.000096,0.000080,0.000067,0.000055,0.000046,0.000038,0.000032,0.000027,0.000022,0.000018,...,4.615618e-07,3.839103e-07,3.193227e-07,2.656011e-07,2.209173e-07,1.837510e-07,1.528375e-07,1.271247e-07,1.057377e-07,8.794878e-08
"(-124.5, 48.0)",0.000366,0.000305,0.000253,0.000211,0.000175,0.000146,0.000121,0.000101,0.000084,0.000070,...,1.753517e-06,1.458512e-06,1.213137e-06,1.009044e-06,8.392859e-07,6.980876e-07,5.806440e-07,4.829586e-07,4.017075e-07,3.341257e-07


In [71]:
epicenters = df.index.to_numpy()
magnitudes = df.columns.to_numpy()

In [ ]:
I = np.ones((g.number_of_edges(), len(epicenters), len(magnitudes)))
for i, e in tqdm(enumerate(g.edges), total=g.number_of_edges()):
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    l = LineString(edge_points)
    for j, epicenter in tqdm(enumerate(epicenters), total=len(epicenters)):
        p = Point(epicenter)
        nearest_point = nearest_points(l, p)[0].coords[0]
        R = distance(lonlat(*epicenter), lonlat(*nearest_point)).km
        for k, M in enumerate(magnitudes):
            intensity = intensity_europe(np.float(M), R)
            if intensity > 1.0:
                I[i,j,k] = intensity

In [73]:
np.save('intensities/usa_99_ds23.npy', I)

# SRLG collection

In [31]:
def remains_connected(g, srlg):
    g.remove_edges_from(srlg)
    return nx.is_connected(g.to_undirected())

def get_minimal_cut_SRLGs(PSRLG_file):
    xtree = ET.parse(PSRLG_file)
    xroot = xtree.getroot().find('PSRLGList')
    cut_srlgs = []
    for fs in tqdm(xroot.iter('PSRLG'), total=len(xroot.findall('PSRLG'))):
    #for fs in tqdm(xroot.iter('Failure_State'), total=len(xroot.findall('Failure_State'))):
        # (edge_id, node_id, node_id)
        #srlg = set([ (int(edge[1]), int(edge[2]), 0) if edge[0] != '24' else (int(edge[1]), int(edge[2]), 1) for edge in re.findall(r"(\d+):\((\d+)\D+(\d+)\D+\)", fs.find('Edges').text)])
        srlg = set([ (int(edge[1]), int(edge[2]), 0) for edge in re.findall(r"(\d+):\((\d+)\D+(\d+)\)", fs.find('Edges').text)])
        probability = float(fs.find('Probability').text.strip())
        #print(fs.find('Edges').text)
        #print(probability, srlg)
        if not remains_connected(g.copy(), srlg) and probability>10**-4:
            added = False
            for i, s in enumerate(cut_srlgs):
                if srlg.issubset(s):
                    cut_srlgs[i] = srlg
                    added = True
                    break
                elif srlg.issuperset(s):
                    added = True
                    break
            if not cut_srlgs or not added:
                cut_srlgs.append(srlg)
    return cut_srlgs

min_cut_srlgs = get_minimal_cut_SRLGs('PSRLGs/italy_995_simple_it6.xml')

In [35]:
len(min_cut_srlgs)

35

In [34]:
# Correcting node order in SRLG edges
edge_list = list(g.edges)
for idx,srlg in enumerate(min_cut_srlgs):
    for l in srlg:
        if l not in edge_list:
            min_cut_srlgs[idx].discard(l)
            min_cut_srlgs[idx].add((l[1],l[0],l[2]))

In [37]:
# Write
with open('min_cut_SRLGs/italy_995_10-4', 'wb') as fp:
    pickle.dump(min_cut_srlgs, fp)

# Read
#with open ('min_cut_SRLGs/italy', 'rb') as fp:
#    cut_srlgs = pickle.load(fp)

In [36]:
min_cut_srlgs

[{(4, 12, 0), (12, 13, 0), (12, 18, 0), (12, 28, 0), (12, 30, 0)},
 {(4, 12, 0), (4, 30, 0), (4, 31, 0)},
 {(4, 31, 0), (29, 31, 0), (30, 31, 0)},
 {(1, 29, 0), (2, 29, 0), (3, 29, 0), (17, 29, 0), (29, 30, 0), (29, 31, 0)},
 {(0, 28, 0), (9, 28, 0), (12, 28, 0), (26, 28, 0)},
 {(3, 30, 0),
  (4, 30, 0),
  (12, 30, 0),
  (18, 30, 0),
  (19, 30, 0),
  (29, 30, 0),
  (30, 31, 0)},
 {(0, 9, 0), (0, 13, 0), (0, 28, 0)},
 {(7, 18, 0), (9, 18, 0), (12, 18, 0), (18, 19, 0), (18, 30, 0)},
 {(1, 3, 0), (1, 16, 0), (1, 17, 0), (1, 29, 0), (17, 29, 0)},
 {(0, 9, 0), (0, 13, 0), (0, 28, 0), (12, 13, 0), (13, 19, 0)},
 {(1, 3, 0), (3, 7, 0), (3, 29, 0), (3, 30, 0)},
 {(0, 9, 0), (9, 18, 0), (9, 19, 0), (9, 26, 0), (9, 28, 0)},
 {(9, 26, 0), (19, 26, 0), (25, 26, 0), (26, 28, 0)},
 {(7, 19, 0),
  (9, 19, 0),
  (13, 19, 0),
  (18, 19, 0),
  (19, 22, 0),
  (19, 23, 0),
  (19, 26, 0),
  (19, 30, 0)},
 {(3, 7, 0), (7, 16, 0), (7, 18, 0), (7, 19, 0), (7, 23, 0)},
 {(1, 16, 0), (7, 16, 0), (10, 16, 0), (1